<a href="https://colab.research.google.com/github/elifmacit/booksearch/blob/main/booksearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import unicodedata #Aksanları kaldırmak için GEREKLİ
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
drive.mount('/content/drive')

# ---- Veritabanına bağlan ----
DB_PATH = "/content/drive/MyDrive/books.db"
conn = sqlite3.connect(DB_PATH)

# --- (SQLite için Aksan Temizleme Fonksiyonu) ---
def strip_accents(s):
    """Veritabanı metinleri için aksanları kaldırır (é -> e)"""
    if not isinstance(s, str):
        return ""
    # 'NFD' ile ayır (örn: é -> e + ´) ve 'Mn' (Aksan/İşaret) kategorisindekileri at
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

# Bu Python fonksiyonunu SQL'de "strip_accents" adıyla kullanılabilir yap
conn.create_function("strip_accents", 1, strip_accents)



# --- (TF-IDF için Aksan Temizleme Ayarı) ---
vectorizer = TfidfVectorizer(
    stop_words="english",
    max_features=50000,
    strip_accents='unicode'  # 'Fénix'i 'fenix' olarak ele al
)



# ---- SQL ile aday kitapları çek (Aksanlara duyarsız ) ----
def get_candidates(query):
    # 1. Sorguyu kelimelere ayır
    keywords = query.lower().split()

    if not keywords:
        return pd.DataFrame(columns=['id', 'title', 'authors', 'publisher', 'average_rating'])

    # 2. Dinamik SQL ve parametre listesi oluştur
    fields = ['title', 'authors', 'publisher']
    where_clauses = []
    params = []

    for key in keywords:
        # Gelen arama terimini de aksansız hale getir
        processed_key = strip_accents(key)
        like_key = f"%{processed_key}%"

        for field in fields:
            # Veritabanındaki alanı da aksansız hale getirerek karşılaştır
            where_clauses.append(f"strip_accents(lower({field})) LIKE ?")
            params.append(like_key)

    # 3. Tüm "OR" şartlarını birleştir
    where_sql = " OR ".join(where_clauses)

    sql = f"""
    SELECT
        rowid AS id,
        COALESCE(title,'') AS title,
        COALESCE(authors,'') AS authors,
        COALESCE(publisher,'') AS publisher,
        COALESCE(average_rating,0) AS average_rating
    FROM books
    WHERE {where_sql}
    LIMIT 200;
    """

    return pd.read_sql(sql, conn, params=params)

# ---- Arama fonksiyonu  ----
def search_books(query, top_k= 20, alpha=0.5):
    df = get_candidates(query)

    if len(df) == 0:
        print(f"❗ '{query}' için veri bulunamadı.")
        return

    # Metin alanını birleştir
    df["text"] = (
        df["title"].str.lower() + " " +
        df["authors"].str.lower() + " " +
        df["publisher"].str.lower()
    )

    # TF-IDF vektörleri oluştur
    # (vectorizer artık 'strip_accents' ayarı ile çalışacak)
    X = vectorizer.fit_transform(df["text"])
    q_vec = vectorizer.transform([query.lower()])

    # Kosinüs benzerliği
    cosine_scores = cosine_similarity(q_vec, X).ravel()

    # Rating'i normalize et (0-1)
    r = df["average_rating"].astype(float).values

    r_range = r.max() - r.min()
    if r_range == 0:
        r_norm = np.zeros_like(r)
    else:
        r_norm = (r - r.min()) / (r_range + 1e-9)

    # Ağırlıklı skor = cosine + rating
    final_score = (1 - alpha) * cosine_scores + alpha * r_norm

    # Sırala
    idx = np.argsort(-final_score)[:top_k]
    results = df.iloc[idx]

    # Sonuç yoksa
    if cosine_scores.max() < 0.01:
        print(f"❗ '{query}' ile ilgili anlamlı sonuç bulunamadı.")
        return

    # Yazdır
    print(f"\n🔎 Arama: '{query}' — Toplam {len(results)} sonuç\n")
    for _, row in results.iterrows():
        print(f"- {row['title']} — {row['authors']} (Rating: {row['average_rating']})")

# ---- Kullanıcı etkileşimi ----
print("📚 Kitap Arama Sistemi — boş Enter = çıkış")
while True:
    q = input("\nSorgu: ")
    if q.strip() == "":
        print("👋 Çıkış yapıldı.")
        break
    search_books(q)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📚 Kitap Arama Sistemi — boş Enter = çıkış
